In [10]:
from utils import *
from atae_lstm import *
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
from torchsummary import summary

In [11]:
TRAIN_FILE_PATH = '../data/absa/SemEval14/abas-pytorch/Laptops_Train.xml.seg'
TEST_FILE_PATH = '../data/absa/SemEval14/abas-pytorch/Laptops_Test_Gold.xml.seg'

In [12]:
BATCH_SIZE = 16
NUM_EPOCHS = 20

In [13]:
tokenizer = build_tokenizer(fnames=[TRAIN_FILE_PATH, TEST_FILE_PATH], 
                            max_seq_len=85,
                            dat_fname='./output/laptop_tokenizer.dat')

loading tokenizer:  ./output/laptop_tokenizer.dat
finish!


In [14]:
trainset = ABSADataset(TRAIN_FILE_PATH, tokenizer)
testset = ABSADataset(TEST_FILE_PATH, tokenizer)

print(len(trainset))
val_len = int(len(trainset) * 0.1)
trainset, valset = random_split(trainset, [len(trainset) - val_len, val_len])
print(len(trainset))

2328
2096


In [15]:
x1 = [1, 2]
x2 = [11, 22, 33, 44]
x_len = [2, 4]
x_len = torch.tensor(x_len)
x1_pad = pad_and_truncate(x1, 4)
x2_pad = pad_and_truncate(x2, 4)
print('x1: ', x1_pad)
print('x2: ', x2_pad)

x_batch = np.vstack((x1_pad, x2_pad))
x_batch = torch.tensor(x_batch)
print('x_batch: ', x_batch)

embedding = nn.Embedding(1000, 3, padding_idx=0)
x_batch_embd = embedding(x_batch)
print('embedding: ', x_batch_embd)

squeeze_embedding = SqueezeEmbedding()
x_batch_squeeze_embd = squeeze_embedding(x_batch_embd, x_len)
print('squeeze embedding: ', x_batch_squeeze_embd)

rnn_test = DynamicRNN(3, 6)
out, (ht, ct) = rnn_test(x_batch_embd, x_len)

x1:  [1 2 0 0]
x2:  [11 22 33 44]
x_batch:  tensor([[ 1,  2,  0,  0],
        [11, 22, 33, 44]])
embedding:  tensor([[[ 1.0753, -0.1936,  0.2285],
         [-1.7786, -0.0876,  0.7390],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000]],

        [[-0.5963,  1.1189, -0.8744],
         [ 0.5256,  0.2629, -0.7533],
         [-0.6376, -0.9655, -0.8938],
         [-0.3643, -1.5764, -0.0581]]], grad_fn=<EmbeddingBackward0>)
squeeze embedding:  tensor([[[ 1.0753, -0.1936,  0.2285],
         [-1.7786, -0.0876,  0.7390],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000]],

        [[-0.5963,  1.1189, -0.8744],
         [ 0.5256,  0.2629, -0.7533],
         [-0.6376, -0.9655, -0.8938],
         [-0.3643, -1.5764, -0.0581]]], grad_fn=<IndexBackward0>)
rnn forward....
x raw:  tensor([[[ 1.0753, -0.1936,  0.2285],
         [-1.7786, -0.0876,  0.7390],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000]],

        [[-0.5963,  1